# ComfyUI Folder Visibility Diagnostic

This notebook helps diagnose why the ComfyUI folder is not visible in JupyterLab's file browser even though ComfyUI is running properly.

## Problem Description
- ComfyUI web interface is accessible and working
- ComfyUI folder is not visible in JupyterLab file browser
- Need to investigate directory structure, permissions, and JupyterLab configuration

## Investigation Steps
We'll systematically check various aspects that could cause this visibility issue.

In [ ]:
import os
import sys
import subprocess
import stat
from pathlib import Path
import json
from datetime import datetime

print(f"Diagnostic started at: {datetime.now()}")
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")

## 1. Check Current Working Directory

First, let's confirm where JupyterLab is running from and what the current working directory is.

In [ ]:
# Check current working directory using multiple methods
print("=== Current Working Directory ===")
print(f"os.getcwd(): {os.getcwd()}")
print(f"os.path.abspath('.'): {os.path.abspath('.')}")
print(f"Path.cwd(): {Path.cwd()}")

# Also check using shell command
print("\n=== Shell pwd command ===")
result = subprocess.run(['pwd'], capture_output=True, text=True, shell=True)
print(f"pwd output: {result.stdout.strip()}")

# Check if we're in a container
print("\n=== Container Environment Check ===")
if os.path.exists('/.dockerenv'):
    print("✓ Running inside Docker container")
else:
    print("Not running inside Docker container")

# Check hostname
hostname_result = subprocess.run(['hostname'], capture_output=True, text=True)
print(f"Hostname: {hostname_result.stdout.strip()}")

## 2. List All Files and Folders

Let's examine the directory structure to see what folders and files are visible.

In [ ]:
# List files and directories in current location
print("=== Directory Listing (Python os.listdir) ===")
current_items = os.listdir('.')
print(f"Items in current directory ({len(current_items)}):")
for item in sorted(current_items):
    item_path = os.path.join('.', item)
    if os.path.isdir(item_path):
        print(f"  📁 {item}/")
    else:
        print(f"  📄 {item}")

print("\n=== Detailed Directory Listing (ls -la) ===")
ls_result = subprocess.run(['ls', '-la'], capture_output=True, text=True)
print(ls_result.stdout)

print("\n=== Look for ComfyUI specifically ===")
# Check if ComfyUI directory exists
comfyui_paths = [
    '/workspace/ComfyUI',
    './ComfyUI',
    '../ComfyUI',
    '/ComfyUI',
    '/workspace/comfy_template/ComfyUI'
]

for path in comfyui_paths:
    if os.path.exists(path):
        print(f"✓ Found ComfyUI at: {path}")
        print(f"  Is directory: {os.path.isdir(path)}")
        print(f"  Absolute path: {os.path.abspath(path)}")
    else:
        print(f"✗ Not found: {path}")

In [ ]:
# Check the /workspace directory specifically
print("=== /workspace Directory Check ===")
workspace_path = '/workspace'
if os.path.exists(workspace_path):
    print(f"✓ /workspace exists")
    print(f"Contents of {workspace_path}:")
    try:
        workspace_items = os.listdir(workspace_path)
        for item in sorted(workspace_items):
            item_path = os.path.join(workspace_path, item)
            if os.path.isdir(item_path):
                print(f"  📁 {item}/")
            else:
                print(f"  📄 {item}")
    except PermissionError:
        print(f"  ❌ Permission denied to list {workspace_path}")
else:
    print(f"✗ /workspace does not exist")

# Use find command to search for ComfyUI
print("\n=== Search for ComfyUI using find command ===")
find_result = subprocess.run(['find', '/', '-name', 'ComfyUI', '-type', 'd', '2>/dev/null'], 
                           capture_output=True, text=True, shell=True)
if find_result.stdout.strip():
    print("Found ComfyUI directories:")
    for line in find_result.stdout.strip().split('\n'):
        print(f"  {line}")
else:
    print("No ComfyUI directories found")

## 3. Verify Folder Permissions

Let's check the permissions of the ComfyUI folder and related directories to see if permission issues are preventing visibility.

In [ ]:
# Check permissions of key directories
print("=== Directory Permissions Check ===")

def check_permissions(path):
    """Check and display permissions for a given path"""
    if not os.path.exists(path):
        print(f"✗ {path} - Does not exist")
        return
    
    try:
        stat_info = os.stat(path)
        mode = stat_info.st_mode
        permissions = stat.filemode(mode)
        uid = stat_info.st_uid
        gid = stat_info.st_gid
        
        print(f"✓ {path}")
        print(f"  Permissions: {permissions}")
        print(f"  Owner UID: {uid}, GID: {gid}")
        print(f"  Is readable: {os.access(path, os.R_OK)}")
        print(f"  Is writable: {os.access(path, os.W_OK)}")
        print(f"  Is executable: {os.access(path, os.X_OK)}")
        
        # Show detailed ls output
        ls_result = subprocess.run(['ls', '-ld', path], capture_output=True, text=True)
        print(f"  ls -ld: {ls_result.stdout.strip()}")
        print()
        
    except Exception as e:
        print(f"❌ Error checking {path}: {e}")
        print()

# Check permissions for important paths
paths_to_check = [
    '.',
    '/workspace',
    '/workspace/ComfyUI',
    '/workspace/comfy_template',
    os.getcwd()
]

# Add any found ComfyUI paths
for path in comfyui_paths:
    if os.path.exists(path):
        paths_to_check.append(path)

# Remove duplicates
paths_to_check = list(set(paths_to_check))

for path in paths_to_check:
    check_permissions(path)

## 4. Check JupyterLab Configuration

Let's examine JupyterLab's configuration to see if there are any settings that might be hiding the ComfyUI folder.

In [ ]:
# Check Jupyter configuration
print("=== JupyterLab Configuration Check ===")

# Get Jupyter config directory
config_result = subprocess.run(['jupyter', '--config-dir'], capture_output=True, text=True)
print(f"Jupyter config directory: {config_result.stdout.strip()}")

# Get Jupyter data directory
data_result = subprocess.run(['jupyter', '--data-dir'], capture_output=True, text=True)
print(f"Jupyter data directory: {data_result.stdout.strip()}")

# Get Jupyter runtime directory
runtime_result = subprocess.run(['jupyter', '--runtime-dir'], capture_output=True, text=True)
print(f"Jupyter runtime directory: {runtime_result.stdout.strip()}")

print("\n=== Jupyter Lab Status ===")
# Check if jupyter lab is running and its configuration
lab_list_result = subprocess.run(['jupyter', 'lab', 'list'], capture_output=True, text=True)
print("Running Jupyter Lab instances:")
print(lab_list_result.stdout)

print("\n=== Check for Jupyter Config Files ===")
config_dir = config_result.stdout.strip()
if os.path.exists(config_dir):
    print(f"Config directory contents:")
    for item in os.listdir(config_dir):
        print(f"  {item}")
        
    # Look for specific config files
    config_files = [
        'jupyter_lab_config.py',
        'jupyter_notebook_config.py',
        'jupyter_server_config.py'
    ]
    
    for config_file in config_files:
        config_path = os.path.join(config_dir, config_file)
        if os.path.exists(config_path):
            print(f"\n✓ Found {config_file}:")
            with open(config_path, 'r') as f:
                content = f.read()
                if content.strip():
                    print(content[:500] + "..." if len(content) > 500 else content)
                else:
                    print("  (File is empty)")
else:
    print(f"Config directory {config_dir} does not exist")

In [ ]:
# Check the Jupyter server's root directory setting
print("=== Jupyter Server Root Directory ===")

# Try to get the current notebook server info
try:
    server_list_result = subprocess.run(['jupyter', 'server', 'list'], capture_output=True, text=True)
    print("Jupyter server list output:")
    print(server_list_result.stdout)
    if server_list_result.stderr:
        print("Errors:")
        print(server_list_result.stderr)
except Exception as e:
    print(f"Could not get server list: {e}")

# Check environment variables that might affect Jupyter
print("\n=== Relevant Environment Variables ===")
env_vars = ['JUPYTER_CONFIG_DIR', 'JUPYTER_DATA_DIR', 'JUPYTER_RUNTIME_DIR', 
           'JUPYTER_PATH', 'PYTHONPATH', 'HOME', 'PWD']

for var in env_vars:
    value = os.environ.get(var, 'Not set')
    print(f"{var}: {value}")

# Check if we can import jupyter lab modules
print("\n=== Jupyter Lab Module Check ===")
try:
    import jupyterlab
    print(f"✓ JupyterLab version: {jupyterlab.__version__}")
except ImportError:
    print("✗ JupyterLab module not found")

try:
    import notebook
    print(f"✓ Notebook version: {notebook.__version__}")
except ImportError:
    print("✗ Notebook module not found")

## 5. Potential Solutions

Based on the diagnostics above, let's try some solutions to make the ComfyUI folder visible in JupyterLab.

In [ ]:
# Solution 1: Create a symbolic link to ComfyUI in the current directory
print("=== Solution 1: Create Symbolic Link ===")

# Find the actual ComfyUI directory
comfyui_actual_path = None
for path in comfyui_paths:
    if os.path.exists(path) and os.path.isdir(path):
        comfyui_actual_path = os.path.abspath(path)
        break

if comfyui_actual_path:
    print(f"Found ComfyUI at: {comfyui_actual_path}")
    
    # Create symbolic link in current directory
    current_dir = os.getcwd()
    symlink_path = os.path.join(current_dir, 'ComfyUI')
    
    if os.path.exists(symlink_path):
        print(f"ComfyUI already exists in current directory: {symlink_path}")
        if os.path.islink(symlink_path):
            print(f"It's a symbolic link pointing to: {os.readlink(symlink_path)}")
    else:
        try:
            os.symlink(comfyui_actual_path, symlink_path)
            print(f"✓ Created symbolic link: {symlink_path} -> {comfyui_actual_path}")
        except Exception as e:
            print(f"❌ Failed to create symbolic link: {e}")
            
            # Try using ln command instead
            print("Trying with ln command...")
            ln_result = subprocess.run(['ln', '-s', comfyui_actual_path, symlink_path], 
                                     capture_output=True, text=True)
            if ln_result.returncode == 0:
                print(f"✓ Created symbolic link with ln command")
            else:
                print(f"❌ ln command failed: {ln_result.stderr}")
else:
    print("❌ Could not find ComfyUI directory to link to")

In [ ]:
# Solution 2: Change to workspace directory
print("\n=== Solution 2: Change Working Directory ===")

workspace_dir = '/workspace'
if os.path.exists(workspace_dir):
    try:
        os.chdir(workspace_dir)
        print(f"✓ Changed working directory to: {os.getcwd()}")
        print("Contents of workspace directory:")
        for item in sorted(os.listdir('.')):
            if os.path.isdir(item):
                print(f"  📁 {item}/")
            else:
                print(f"  📄 {item}")
    except Exception as e:
        print(f"❌ Failed to change to workspace directory: {e}")
else:
    print(f"❌ Workspace directory {workspace_dir} does not exist")

# Check if ComfyUI is now visible
if os.path.exists('ComfyUI'):
    print("\n✓ ComfyUI is now visible in current directory!")
else:
    print("\n❌ ComfyUI still not visible in current directory")

In [ ]:
# Solution 3: Direct access methods
print("\n=== Solution 3: Direct Access Methods ===")

# Method 1: Use pathlib to create a path object
if comfyui_actual_path:
    from pathlib import Path
    comfyui_path = Path(comfyui_actual_path)
    print(f"ComfyUI Path object: {comfyui_path}")
    print(f"ComfyUI exists: {comfyui_path.exists()}")
    print(f"ComfyUI is directory: {comfyui_path.is_dir()}")
    
    if comfyui_path.exists():
        print("\nComfyUI directory contents:")
        try:
            for item in comfyui_path.iterdir():
                if item.is_dir():
                    print(f"  📁 {item.name}/")
                else:
                    print(f"  📄 {item.name}")
        except PermissionError:
            print("  ❌ Permission denied to list ComfyUI contents")

# Method 2: Add to Python path
print("\n=== Adding ComfyUI to Python Path ===")
if comfyui_actual_path and comfyui_actual_path not in sys.path:
    sys.path.append(comfyui_actual_path)
    print(f"✓ Added {comfyui_actual_path} to Python path")
else:
    print("ComfyUI path already in Python path or not found")

print(f"\nCurrent Python path includes:")
for path in sys.path:
    if 'ComfyUI' in path or 'workspace' in path:
        print(f"  {path}")

## 6. Recommendations and Next Steps

Based on the diagnostic results above, here are the recommended next steps:

### If ComfyUI was found but not visible in JupyterLab:
1. **Symbolic Link**: Create a symbolic link in the JupyterLab working directory
2. **Change Directory**: Navigate JupyterLab to the `/workspace` directory
3. **Restart JupyterLab**: Sometimes a restart is needed to refresh the file browser

### If ComfyUI was not found:
1. **Check Installation**: Verify that the ComfyUI installation script completed successfully
2. **Check Docker Volumes**: Ensure Docker volumes are properly mounted
3. **Manual Installation**: Run the installation script manually

### File Browser Refresh:
Try refreshing the JupyterLab file browser by clicking the refresh button or pressing F5.

### Terminal Access:
Use the JupyterLab terminal to navigate directly to ComfyUI:
```bash
cd /workspace/ComfyUI
ls -la
```

In [ ]:
# Final status check
print("=== Final Status Check ===")
print(f"Current working directory: {os.getcwd()}")
print(f"ComfyUI visible in current directory: {os.path.exists('ComfyUI')}")
print(f"ComfyUI visible in /workspace: {os.path.exists('/workspace/ComfyUI')}")

# Check if we can access ComfyUI files
if os.path.exists('/workspace/ComfyUI'):
    comfyui_main = '/workspace/ComfyUI/main.py'
    if os.path.exists(comfyui_main):
        print(f"✓ ComfyUI main.py found at {comfyui_main}")
    else:
        print(f"❌ ComfyUI main.py not found")

print("\n=== Instructions for Manual Access ===")
print("If ComfyUI is still not visible in the file browser, you can:")
print("1. Use the terminal tab in JupyterLab")
print("2. Navigate to /workspace/ComfyUI")
print("3. Use the symbolic link created above (if successful)")
print("4. Restart JupyterLab and check again")

print(f"\nDiagnostic completed at: {datetime.now()}")